## Read data

In [1]:
import pandas as pd

# Read the data
X = pd.read_csv('./input/train.csv', index_col='Id')
X_test = pd.read_csv('./input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

## Split data on train and test

In [2]:
from sklearn.model_selection import train_test_split

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

## Create preprocessor

#### Get low cardinality and numeric cols

In [3]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and 
                        X_train[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

#### Preprocessing data

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#### Create preprocessor

In [5]:
from sklearn.compose import ColumnTransformer

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_cols),
        ('cat', categorical_transformer, low_cardinality_cols)
    ])

## Choose model

[CatBoost regressor parameters](https://catboost.ai/docs/concepts/python-reference_catboostregressor.html)

In [27]:
from catboost import CatBoostRegressor

# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=5000,
                            learning_rate=0.01,
                            max_depth=5,
                            silent=True)

## Create pipeline

In [28]:
# Bundle preprocessing and modeling code in a pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

## Train model use cross validation

In [29]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

kfold = KFold(n_splits=5, random_state=42, shuffle=True)
scores = -1 * cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring='neg_mean_absolute_error')

print(scores)
print("Average MAE score:", scores.mean())

0:	learn: 193112.5840982	total: 30.2ms	remaining: 2m 31s
1:	learn: 191375.6765645	total: 56ms	remaining: 2m 19s
2:	learn: 189653.5370509	total: 82.2ms	remaining: 2m 16s
3:	learn: 187989.0965103	total: 113ms	remaining: 2m 21s
4:	learn: 186396.5919618	total: 140ms	remaining: 2m 19s
5:	learn: 184775.0017057	total: 167ms	remaining: 2m 18s
6:	learn: 183155.1343565	total: 195ms	remaining: 2m 18s
7:	learn: 181510.5305156	total: 222ms	remaining: 2m 18s
8:	learn: 179961.9727764	total: 256ms	remaining: 2m 22s
9:	learn: 178384.6042998	total: 285ms	remaining: 2m 22s
10:	learn: 176811.6617113	total: 314ms	remaining: 2m 22s
11:	learn: 175310.1385996	total: 340ms	remaining: 2m 21s
12:	learn: 173789.7561465	total: 367ms	remaining: 2m 20s
13:	learn: 172277.0660952	total: 394ms	remaining: 2m 20s
14:	learn: 170802.6136561	total: 420ms	remaining: 2m 19s
15:	learn: 169317.7365854	total: 449ms	remaining: 2m 19s
16:	learn: 167864.7874786	total: 476ms	remaining: 2m 19s
17:	learn: 166386.8390659	total: 502ms	r

148:	learn: 60974.6821624	total: 4.09s	remaining: 2m 13s
149:	learn: 60612.8558523	total: 4.12s	remaining: 2m 13s
150:	learn: 60239.4580088	total: 4.14s	remaining: 2m 13s
151:	learn: 59854.3245472	total: 4.17s	remaining: 2m 13s
152:	learn: 59470.8170515	total: 4.2s	remaining: 2m 13s
153:	learn: 59109.1324758	total: 4.23s	remaining: 2m 13s
154:	learn: 58740.0951895	total: 4.25s	remaining: 2m 13s
155:	learn: 58386.3056353	total: 4.28s	remaining: 2m 12s
156:	learn: 58032.6916183	total: 4.31s	remaining: 2m 12s
157:	learn: 57678.7545269	total: 4.34s	remaining: 2m 12s
158:	learn: 57329.2911374	total: 4.36s	remaining: 2m 12s
159:	learn: 56989.7815413	total: 4.39s	remaining: 2m 12s
160:	learn: 56671.5292309	total: 4.41s	remaining: 2m 12s
161:	learn: 56332.7269635	total: 4.44s	remaining: 2m 12s
162:	learn: 56000.2506257	total: 4.47s	remaining: 2m 12s
163:	learn: 55665.2427338	total: 4.5s	remaining: 2m 12s
164:	learn: 55339.9209233	total: 4.52s	remaining: 2m 12s
165:	learn: 55022.3106751	total: 

294:	learn: 33000.6048745	total: 8.2s	remaining: 2m 10s
295:	learn: 32915.2305945	total: 8.22s	remaining: 2m 10s
296:	learn: 32823.2000943	total: 8.26s	remaining: 2m 10s
297:	learn: 32750.4422453	total: 8.29s	remaining: 2m 10s
298:	learn: 32661.0748716	total: 8.31s	remaining: 2m 10s
299:	learn: 32586.9610562	total: 8.34s	remaining: 2m 10s
300:	learn: 32501.1292859	total: 8.37s	remaining: 2m 10s
301:	learn: 32418.0374335	total: 8.39s	remaining: 2m 10s
302:	learn: 32346.7717052	total: 8.42s	remaining: 2m 10s
303:	learn: 32263.7055912	total: 8.45s	remaining: 2m 10s
304:	learn: 32175.8858157	total: 8.48s	remaining: 2m 10s
305:	learn: 32099.2212263	total: 8.51s	remaining: 2m 10s
306:	learn: 32027.3325150	total: 8.53s	remaining: 2m 10s
307:	learn: 31969.9318374	total: 8.56s	remaining: 2m 10s
308:	learn: 31900.8866821	total: 8.59s	remaining: 2m 10s
309:	learn: 31848.1226244	total: 8.61s	remaining: 2m 10s
310:	learn: 31760.1350197	total: 8.64s	remaining: 2m 10s
311:	learn: 31697.1416084	total:

441:	learn: 26102.6906526	total: 12.2s	remaining: 2m 6s
442:	learn: 26067.0012531	total: 12.3s	remaining: 2m 6s
443:	learn: 26055.0206074	total: 12.3s	remaining: 2m 5s
444:	learn: 26041.4975324	total: 12.3s	remaining: 2m 5s
445:	learn: 26000.2023723	total: 12.3s	remaining: 2m 5s
446:	learn: 25988.1959910	total: 12.4s	remaining: 2m 5s
447:	learn: 25950.4873719	total: 12.4s	remaining: 2m 5s
448:	learn: 25923.5752988	total: 12.4s	remaining: 2m 5s
449:	learn: 25897.7347287	total: 12.4s	remaining: 2m 5s
450:	learn: 25876.8571553	total: 12.5s	remaining: 2m 5s
451:	learn: 25855.5836862	total: 12.5s	remaining: 2m 5s
452:	learn: 25838.4095620	total: 12.5s	remaining: 2m 5s
453:	learn: 25823.8436167	total: 12.5s	remaining: 2m 5s
454:	learn: 25800.7580709	total: 12.6s	remaining: 2m 5s
455:	learn: 25771.4294435	total: 12.6s	remaining: 2m 5s
456:	learn: 25745.5319946	total: 12.6s	remaining: 2m 5s
457:	learn: 25722.4509120	total: 12.7s	remaining: 2m 5s
458:	learn: 25691.1683118	total: 12.7s	remaining

588:	learn: 23339.3222974	total: 16.2s	remaining: 2m 1s
589:	learn: 23314.3465281	total: 16.2s	remaining: 2m 1s
590:	learn: 23299.5845216	total: 16.3s	remaining: 2m 1s
591:	learn: 23295.7947720	total: 16.3s	remaining: 2m 1s
592:	learn: 23277.1954539	total: 16.3s	remaining: 2m 1s
593:	learn: 23258.6678449	total: 16.4s	remaining: 2m 1s
594:	learn: 23251.4062256	total: 16.4s	remaining: 2m 1s
595:	learn: 23238.7059141	total: 16.4s	remaining: 2m 1s
596:	learn: 23219.5854818	total: 16.4s	remaining: 2m 1s
597:	learn: 23213.7143494	total: 16.5s	remaining: 2m 1s
598:	learn: 23203.0750912	total: 16.5s	remaining: 2m 1s
599:	learn: 23182.5623775	total: 16.5s	remaining: 2m 1s
600:	learn: 23168.2881739	total: 16.5s	remaining: 2m 1s
601:	learn: 23154.2570015	total: 16.6s	remaining: 2m 1s
602:	learn: 23150.5527391	total: 16.6s	remaining: 2m 1s
603:	learn: 23132.2658732	total: 16.6s	remaining: 2m
604:	learn: 23113.8123829	total: 16.6s	remaining: 2m
605:	learn: 23106.6191496	total: 16.7s	remaining: 2m
6

...

4333:	learn: 14597.7995992	total: 1m 59s	remaining: 18.3s
4334:	learn: 14595.8118841	total: 1m 59s	remaining: 18.3s
4335:	learn: 14595.2793320	total: 1m 59s	remaining: 18.2s
4336:	learn: 14594.6485822	total: 1m 59s	remaining: 18.2s
4337:	learn: 14594.6175583	total: 1m 59s	remaining: 18.2s
4338:	learn: 14594.5622687	total: 1m 59s	remaining: 18.2s
4339:	learn: 14593.9908531	total: 1m 59s	remaining: 18.1s
4340:	learn: 14593.9301403	total: 1m 59s	remaining: 18.1s
4341:	learn: 14592.9812853	total: 1m 59s	remaining: 18.1s
4342:	learn: 14591.8125913	total: 1m 59s	remaining: 18s
4343:	learn: 14584.8912303	total: 1m 59s	remaining: 18s
4344:	learn: 14584.7947624	total: 1m 59s	remaining: 18s
4345:	learn: 14584.4255214	total: 1m 59s	remaining: 18s
4346:	learn: 14584.0554439	total: 1m 59s	remaining: 17.9s
4347:	learn: 14583.9486393	total: 1m 59s	remaining: 17.9s
4348:	learn: 14583.2750845	total: 1m 59s	remaining: 17.9s
4349:	learn: 14583.1612402	total: 1m 59s	remaining: 17.9s
4350:	learn: 14583.055

4479:	learn: 14445.1210422	total: 2m 3s	remaining: 14.3s
4480:	learn: 14441.2215225	total: 2m 3s	remaining: 14.3s
4481:	learn: 14438.2764310	total: 2m 3s	remaining: 14.2s
4482:	learn: 14435.9768820	total: 2m 3s	remaining: 14.2s
4483:	learn: 14435.0029773	total: 2m 3s	remaining: 14.2s
4484:	learn: 14434.0803147	total: 2m 3s	remaining: 14.1s
4485:	learn: 14430.1987602	total: 2m 3s	remaining: 14.1s
4486:	learn: 14429.7628080	total: 2m 3s	remaining: 14.1s
4487:	learn: 14427.7679635	total: 2m 3s	remaining: 14.1s
4488:	learn: 14427.5573876	total: 2m 3s	remaining: 14s
4489:	learn: 14427.5163720	total: 2m 3s	remaining: 14s
4490:	learn: 14424.9264965	total: 2m 3s	remaining: 14s
4491:	learn: 14424.0341233	total: 2m 3s	remaining: 13.9s
4492:	learn: 14422.9434154	total: 2m 3s	remaining: 13.9s
4493:	learn: 14422.5668719	total: 2m 3s	remaining: 13.9s
4494:	learn: 14422.4182708	total: 2m 3s	remaining: 13.9s
4495:	learn: 14416.8989719	total: 2m 3s	remaining: 13.8s
4496:	learn: 14416.3680109	total: 2m 

4627:	learn: 14307.3683605	total: 2m 7s	remaining: 10.2s
4628:	learn: 14307.1751331	total: 2m 7s	remaining: 10.2s
4629:	learn: 14305.1179110	total: 2m 7s	remaining: 10.2s
4630:	learn: 14305.0091834	total: 2m 7s	remaining: 10.1s
4631:	learn: 14304.8765710	total: 2m 7s	remaining: 10.1s
4632:	learn: 14303.9318824	total: 2m 7s	remaining: 10.1s
4633:	learn: 14303.5828169	total: 2m 7s	remaining: 10s
4634:	learn: 14303.0161478	total: 2m 7s	remaining: 10s
4635:	learn: 14302.3617221	total: 2m 7s	remaining: 9.99s
4636:	learn: 14301.8674967	total: 2m 7s	remaining: 9.96s
4637:	learn: 14301.3935398	total: 2m 7s	remaining: 9.94s
4638:	learn: 14298.0908371	total: 2m 7s	remaining: 9.91s
4639:	learn: 14298.0353400	total: 2m 7s	remaining: 9.88s
4640:	learn: 14297.5813364	total: 2m 7s	remaining: 9.85s
4641:	learn: 14297.5449189	total: 2m 7s	remaining: 9.82s
4642:	learn: 14297.2774370	total: 2m 7s	remaining: 9.8s
4643:	learn: 14295.6222965	total: 2m 7s	remaining: 9.77s
4644:	learn: 14293.3696256	total: 2m

4774:	learn: 14180.5412096	total: 2m 11s	remaining: 6.18s
4775:	learn: 14180.3152814	total: 2m 11s	remaining: 6.15s
4776:	learn: 14180.2320175	total: 2m 11s	remaining: 6.13s
4777:	learn: 14180.1378052	total: 2m 11s	remaining: 6.1s
4778:	learn: 14179.8432374	total: 2m 11s	remaining: 6.07s
4779:	learn: 14177.6642925	total: 2m 11s	remaining: 6.04s
4780:	learn: 14175.6652330	total: 2m 11s	remaining: 6.01s
4781:	learn: 14174.8746575	total: 2m 11s	remaining: 5.99s
4782:	learn: 14174.7484080	total: 2m 11s	remaining: 5.96s
4783:	learn: 14174.4971409	total: 2m 11s	remaining: 5.93s
4784:	learn: 14174.3875628	total: 2m 11s	remaining: 5.91s
4785:	learn: 14173.7501958	total: 2m 11s	remaining: 5.88s
4786:	learn: 14173.4861938	total: 2m 11s	remaining: 5.85s
4787:	learn: 14173.3342895	total: 2m 11s	remaining: 5.82s
4788:	learn: 14173.0182891	total: 2m 11s	remaining: 5.8s
4789:	learn: 14172.9401951	total: 2m 11s	remaining: 5.77s
4790:	learn: 14172.6139744	total: 2m 11s	remaining: 5.74s
4791:	learn: 141

4922:	learn: 14082.7884364	total: 2m 15s	remaining: 2.12s
4923:	learn: 14082.6861932	total: 2m 15s	remaining: 2.09s
4924:	learn: 14082.1203542	total: 2m 15s	remaining: 2.06s
4925:	learn: 14078.9921444	total: 2m 15s	remaining: 2.04s
4926:	learn: 14078.2391848	total: 2m 15s	remaining: 2.01s
4927:	learn: 14078.1183300	total: 2m 15s	remaining: 1.98s
4928:	learn: 14077.7531863	total: 2m 15s	remaining: 1.95s
4929:	learn: 14077.6298016	total: 2m 15s	remaining: 1.93s
4930:	learn: 14077.3559103	total: 2m 15s	remaining: 1.9s
4931:	learn: 14076.9692689	total: 2m 15s	remaining: 1.87s
4932:	learn: 14076.6581255	total: 2m 15s	remaining: 1.84s
4933:	learn: 14076.6085924	total: 2m 15s	remaining: 1.82s
4934:	learn: 14076.5457542	total: 2m 15s	remaining: 1.79s
4935:	learn: 14074.9250683	total: 2m 15s	remaining: 1.76s
4936:	learn: 14071.4067542	total: 2m 15s	remaining: 1.73s
4937:	learn: 14071.3013449	total: 2m 15s	remaining: 1.71s
4938:	learn: 14069.9172986	total: 2m 15s	remaining: 1.68s
4939:	learn: 14

## Tunning model

#### Get pipeline parameter list

In [33]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'preprocessor', 'model', 'preprocessor__n_jobs', 'preprocessor__remainder', 'preprocessor__sparse_threshold', 'preprocessor__transformer_weights', 'preprocessor__transformers', 'preprocessor__verbose', 'preprocessor__num', 'preprocessor__cat', 'preprocessor__num__add_indicator', 'preprocessor__num__copy', 'preprocessor__num__fill_value', 'preprocessor__num__missing_values', 'preprocessor__num__strategy', 'preprocessor__num__verbose', 'preprocessor__cat__memory', 'preprocessor__cat__steps', 'preprocessor__cat__verbose', 'preprocessor__cat__imputer', 'preprocessor__cat__onehot', 'preprocessor__cat__imputer__add_indicator', 'preprocessor__cat__imputer__copy', 'preprocessor__cat__imputer__fill_value', 'preprocessor__cat__imputer__missing_values', 'preprocessor__cat__imputer__strategy', 'preprocessor__cat__imputer__verbose', 'preprocessor__cat__onehot__categorical_features', 'preprocessor__cat__onehot__categories', 'preprocessor__cat__onehot__drop', 

#### Search best hyperparameters using [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

[CatBoost parameters tunning](https://catboost.ai/docs/concepts/parameter-tuning.html)

In [35]:
from sklearn.model_selection import GridSearchCV

test_params = {
    'model__iterations': [2],
    'model__max_depth': [2, 4, 6],
}

my_model = GridSearchCV(estimator=pipeline, param_grid=test_params)
my_model.fit(X_train, y_train)

print(my_model.best_params_)
print(my_model.best_score_)

d:\kagglelearn\venv\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


0:	learn: 196481.0463310	total: 26.9ms	remaining: 26.9ms
1:	learn: 194788.2627899	total: 41.1ms	remaining: 0us
0:	learn: 190030.7440026	total: 8.66ms	remaining: 8.66ms
1:	learn: 188434.6992793	total: 17.2ms	remaining: 0us
0:	learn: 199402.8919637	total: 8.06ms	remaining: 8.06ms
1:	learn: 197736.8590589	total: 16.1ms	remaining: 0us
0:	learn: 196485.2416147	total: 15.2ms	remaining: 15.2ms
1:	learn: 194722.9883589	total: 30.5ms	remaining: 0us
0:	learn: 190035.0446649	total: 43.1ms	remaining: 43.1ms
1:	learn: 188435.9495240	total: 66.5ms	remaining: 0us
0:	learn: 199420.4267899	total: 44.4ms	remaining: 44.4ms
1:	learn: 197650.1908152	total: 74.5ms	remaining: 0us
0:	learn: 196527.4912070	total: 23ms	remaining: 23ms
1:	learn: 194849.5464374	total: 47.5ms	remaining: 0us
0:	learn: 190046.3173655	total: 23.9ms	remaining: 23.9ms
1:	learn: 188406.7823656	total: 48.8ms	remaining: 0us
0:	learn: 199479.4980051	total: 24ms	remaining: 24ms
1:	learn: 197739.9406478	total: 49.3ms	remaining: 0us
0:	learn:

## Predict on test data and save

#### Fit model

In [30]:
pipeline.fit(X_train, y_train)

0:	learn: 195356.9151328	total: 32.4ms	remaining: 2m 41s
1:	learn: 193609.6693368	total: 69ms	remaining: 2m 52s
2:	learn: 191902.8505378	total: 129ms	remaining: 3m 35s
3:	learn: 190228.0955142	total: 172ms	remaining: 3m 34s
4:	learn: 188526.9257860	total: 206ms	remaining: 3m 26s
5:	learn: 186849.8004810	total: 250ms	remaining: 3m 27s
6:	learn: 185292.0502138	total: 288ms	remaining: 3m 25s
7:	learn: 183679.9093406	total: 311ms	remaining: 3m 14s
8:	learn: 182063.3374436	total: 332ms	remaining: 3m 3s
9:	learn: 180440.3760135	total: 366ms	remaining: 3m 2s
10:	learn: 178849.4271445	total: 403ms	remaining: 3m 2s
11:	learn: 177319.8176421	total: 445ms	remaining: 3m 5s
12:	learn: 175788.0567813	total: 480ms	remaining: 3m 4s
13:	learn: 174252.0176478	total: 516ms	remaining: 3m 3s
14:	learn: 172748.7763659	total: 561ms	remaining: 3m 6s
15:	learn: 171230.6425239	total: 616ms	remaining: 3m 11s
16:	learn: 169712.1439870	total: 655ms	remaining: 3m 12s
17:	learn: 168218.7965218	total: 691ms	remaining

147:	learn: 62030.9591201	total: 5.59s	remaining: 3m 3s
148:	learn: 61645.9401671	total: 5.62s	remaining: 3m 2s
149:	learn: 61250.3891917	total: 5.66s	remaining: 3m 2s
150:	learn: 60867.2960992	total: 5.69s	remaining: 3m 2s
151:	learn: 60487.8118212	total: 5.73s	remaining: 3m 2s
152:	learn: 60111.8944387	total: 5.76s	remaining: 3m 2s
153:	learn: 59752.4657071	total: 5.8s	remaining: 3m 2s
154:	learn: 59412.9117684	total: 5.83s	remaining: 3m 2s
155:	learn: 59048.0507407	total: 5.87s	remaining: 3m 2s
156:	learn: 58692.5780715	total: 5.9s	remaining: 3m 2s
157:	learn: 58349.2803128	total: 5.94s	remaining: 3m 2s
158:	learn: 57999.1154381	total: 5.98s	remaining: 3m 1s
159:	learn: 57638.9773836	total: 6.01s	remaining: 3m 1s
160:	learn: 57291.8835462	total: 6.04s	remaining: 3m 1s
161:	learn: 56948.8344128	total: 6.08s	remaining: 3m 1s
162:	learn: 56622.5471015	total: 6.12s	remaining: 3m 1s
163:	learn: 56271.2355532	total: 6.16s	remaining: 3m 1s
164:	learn: 55967.9531996	total: 6.2s	remaining: 3

295:	learn: 33066.0923272	total: 11.1s	remaining: 2m 56s
296:	learn: 32975.3752407	total: 11.2s	remaining: 2m 56s
297:	learn: 32903.2329243	total: 11.2s	remaining: 2m 56s
298:	learn: 32819.7059896	total: 11.2s	remaining: 2m 56s
299:	learn: 32757.7078509	total: 11.3s	remaining: 2m 56s
300:	learn: 32689.2814706	total: 11.3s	remaining: 2m 56s
301:	learn: 32623.8057805	total: 11.3s	remaining: 2m 56s
302:	learn: 32542.9578812	total: 11.4s	remaining: 2m 56s
303:	learn: 32464.6934140	total: 11.4s	remaining: 2m 56s
304:	learn: 32382.2740556	total: 11.5s	remaining: 2m 56s
305:	learn: 32308.3014447	total: 11.5s	remaining: 2m 56s
306:	learn: 32234.1264700	total: 11.5s	remaining: 2m 56s
307:	learn: 32157.5925499	total: 11.6s	remaining: 2m 56s
308:	learn: 32096.7247675	total: 11.6s	remaining: 2m 55s
309:	learn: 32022.1620924	total: 11.6s	remaining: 2m 55s
310:	learn: 31958.5951317	total: 11.7s	remaining: 2m 55s
311:	learn: 31884.1240228	total: 11.7s	remaining: 2m 55s
312:	learn: 31821.9136329	total

440:	learn: 26469.3315944	total: 16.5s	remaining: 2m 50s
441:	learn: 26439.1919789	total: 16.6s	remaining: 2m 50s
442:	learn: 26412.2586667	total: 16.6s	remaining: 2m 50s
443:	learn: 26374.9859097	total: 16.6s	remaining: 2m 50s
444:	learn: 26349.0053136	total: 16.7s	remaining: 2m 50s
445:	learn: 26322.6994214	total: 16.7s	remaining: 2m 50s
446:	learn: 26299.1916571	total: 16.7s	remaining: 2m 50s
447:	learn: 26259.9401548	total: 16.8s	remaining: 2m 50s
448:	learn: 26238.0146577	total: 16.8s	remaining: 2m 50s
449:	learn: 26206.1314478	total: 16.8s	remaining: 2m 50s
450:	learn: 26188.6735936	total: 16.9s	remaining: 2m 50s
451:	learn: 26169.9704804	total: 16.9s	remaining: 2m 50s
452:	learn: 26122.9875535	total: 17s	remaining: 2m 50s
453:	learn: 26105.2022188	total: 17s	remaining: 2m 50s
454:	learn: 26085.0715482	total: 17s	remaining: 2m 50s
455:	learn: 26053.0032765	total: 17.1s	remaining: 2m 50s
456:	learn: 26033.8317516	total: 17.1s	remaining: 2m 50s
457:	learn: 26001.5756689	total: 17.1

589:	learn: 23702.8169553	total: 22.2s	remaining: 2m 45s
590:	learn: 23694.0892990	total: 22.2s	remaining: 2m 45s
591:	learn: 23683.0255256	total: 22.3s	remaining: 2m 45s
592:	learn: 23673.3349148	total: 22.3s	remaining: 2m 45s
593:	learn: 23659.6916363	total: 22.3s	remaining: 2m 45s
594:	learn: 23652.2149181	total: 22.4s	remaining: 2m 45s
595:	learn: 23644.5866297	total: 22.4s	remaining: 2m 45s
596:	learn: 23626.0339189	total: 22.4s	remaining: 2m 45s
597:	learn: 23615.0121553	total: 22.5s	remaining: 2m 45s
598:	learn: 23593.6405877	total: 22.5s	remaining: 2m 45s
599:	learn: 23582.6142981	total: 22.6s	remaining: 2m 45s
600:	learn: 23562.8298566	total: 22.6s	remaining: 2m 45s
601:	learn: 23554.8007312	total: 22.6s	remaining: 2m 45s
602:	learn: 23539.5627064	total: 22.7s	remaining: 2m 45s
603:	learn: 23524.0872775	total: 22.7s	remaining: 2m 45s
604:	learn: 23507.4081939	total: 22.8s	remaining: 2m 45s
605:	learn: 23504.1240505	total: 22.8s	remaining: 2m 45s
606:	learn: 23494.5139738	total

...

4473:	learn: 16140.2581933	total: 2m 45s	remaining: 19.4s
4474:	learn: 16140.2458758	total: 2m 45s	remaining: 19.4s
4475:	learn: 16140.2350048	total: 2m 45s	remaining: 19.3s
4476:	learn: 16140.1886808	total: 2m 45s	remaining: 19.3s
4477:	learn: 16140.0944275	total: 2m 45s	remaining: 19.3s
4478:	learn: 16137.3514035	total: 2m 45s	remaining: 19.2s
4479:	learn: 16137.3478398	total: 2m 45s	remaining: 19.2s
4480:	learn: 16137.3311033	total: 2m 45s	remaining: 19.1s
4481:	learn: 16137.1804187	total: 2m 45s	remaining: 19.1s
4482:	learn: 16137.0697836	total: 2m 45s	remaining: 19.1s
4483:	learn: 16136.9259830	total: 2m 45s	remaining: 19s
4484:	learn: 16136.9100098	total: 2m 45s	remaining: 19s
4485:	learn: 16136.8750086	total: 2m 45s	remaining: 19s
4486:	learn: 16136.8716860	total: 2m 45s	remaining: 18.9s
4487:	learn: 16136.6662985	total: 2m 45s	remaining: 18.9s
4488:	learn: 16136.6514107	total: 2m 45s	remaining: 18.9s
4489:	learn: 16136.6405425	total: 2m 45s	remaining: 18.8s
4490:	learn: 16136.6

4615:	learn: 16082.2932681	total: 2m 50s	remaining: 14.2s
4616:	learn: 16082.1518480	total: 2m 50s	remaining: 14.1s
4617:	learn: 16082.0439037	total: 2m 50s	remaining: 14.1s
4618:	learn: 16082.0270842	total: 2m 50s	remaining: 14.1s
4619:	learn: 16081.9972333	total: 2m 50s	remaining: 14s
4620:	learn: 16081.9139403	total: 2m 50s	remaining: 14s
4621:	learn: 16081.8840861	total: 2m 50s	remaining: 13.9s
4622:	learn: 16077.4354776	total: 2m 50s	remaining: 13.9s
4623:	learn: 16077.3677005	total: 2m 50s	remaining: 13.9s
4624:	learn: 16072.1953605	total: 2m 50s	remaining: 13.8s
4625:	learn: 16072.1925098	total: 2m 50s	remaining: 13.8s
4626:	learn: 16071.3995380	total: 2m 50s	remaining: 13.8s
4627:	learn: 16071.3960337	total: 2m 50s	remaining: 13.7s
4628:	learn: 16071.3932619	total: 2m 50s	remaining: 13.7s
4629:	learn: 16071.3666354	total: 2m 50s	remaining: 13.6s
4630:	learn: 16071.3472610	total: 2m 50s	remaining: 13.6s
4631:	learn: 16071.2260794	total: 2m 50s	remaining: 13.6s
4632:	learn: 16071

4759:	learn: 16011.1058958	total: 2m 55s	remaining: 8.85s
4760:	learn: 16010.9647449	total: 2m 55s	remaining: 8.82s
4761:	learn: 16010.9360996	total: 2m 55s	remaining: 8.78s
4762:	learn: 16010.3062085	total: 2m 55s	remaining: 8.74s
4763:	learn: 16009.9177533	total: 2m 55s	remaining: 8.71s
4764:	learn: 16007.3267107	total: 2m 55s	remaining: 8.67s
4765:	learn: 16007.3043630	total: 2m 55s	remaining: 8.63s
4766:	learn: 16007.2787946	total: 2m 55s	remaining: 8.6s
4767:	learn: 16006.0977798	total: 2m 55s	remaining: 8.56s
4768:	learn: 16006.0787626	total: 2m 55s	remaining: 8.52s
4769:	learn: 16005.9839935	total: 2m 56s	remaining: 8.49s
4770:	learn: 16005.9600896	total: 2m 56s	remaining: 8.45s
4771:	learn: 16005.9530092	total: 2m 56s	remaining: 8.41s
4772:	learn: 16005.8532684	total: 2m 56s	remaining: 8.38s
4773:	learn: 16005.8162872	total: 2m 56s	remaining: 8.34s
4774:	learn: 16005.7816321	total: 2m 56s	remaining: 8.3s
4775:	learn: 16005.7794464	total: 2m 56s	remaining: 8.26s
4776:	learn: 160

4907:	learn: 15918.7804542	total: 3m 1s	remaining: 3.4s
4908:	learn: 15918.6224245	total: 3m 1s	remaining: 3.36s
4909:	learn: 15918.6069074	total: 3m 1s	remaining: 3.32s
4910:	learn: 15918.5073797	total: 3m 1s	remaining: 3.29s
4911:	learn: 15918.4139502	total: 3m 1s	remaining: 3.25s
4912:	learn: 15918.2367426	total: 3m 1s	remaining: 3.21s
4913:	learn: 15911.9031354	total: 3m 1s	remaining: 3.18s
4914:	learn: 15911.7879692	total: 3m 1s	remaining: 3.14s
4915:	learn: 15911.1504102	total: 3m 1s	remaining: 3.1s
4916:	learn: 15911.0481341	total: 3m 1s	remaining: 3.07s
4917:	learn: 15910.9876749	total: 3m 1s	remaining: 3.03s
4918:	learn: 15910.8479615	total: 3m 1s	remaining: 2.99s
4919:	learn: 15910.8084091	total: 3m 1s	remaining: 2.96s
4920:	learn: 15907.2703823	total: 3m 1s	remaining: 2.92s
4921:	learn: 15907.2416787	total: 3m 1s	remaining: 2.88s
4922:	learn: 15907.1615155	total: 3m 1s	remaining: 2.84s
4923:	learn: 15907.1361404	total: 3m 1s	remaining: 2.81s
4924:	learn: 15905.9310860	total:

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0),
                                                  ['MSSubClass', 'LotFrontage',
                                                   'LotArea', 'OverallQual',
                                                   'OverallCo

#### Predict

In [31]:
# Predict on test data
preds_test = pipeline.predict(X_test)

#### Save predictions

In [32]:
# Save predictions in format used for competition scoring
output = pd.DataFrame({'Id': X_test.index, 'SalePrice': preds_test})
output.to_csv('./output/submission.csv', index=False)